Before we begin, let's execute the cell below to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above. If all goes well, you should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

## Learning objectives
The **goal** of this lab is to:

- Learn how to find bottleneck and performance limiters using Nsight tools
- Learn about three levels of parallelism in OpenACC
- Learn how to use clauses to extract more parallelism in loops

In this section, we will optimize the parallel [RDF](../serial/rdf_overview.ipynb) application using OpenACC. Before we begin, feel free to have a look at the parallel version of the code and inspect it once again. 

[RDF Parallel Code](../../source_code/openacc/SOLUTION/rdf_data_directive.f90)

Now, let's compile, and profile it with Nsight Systems first.

In [ ]:
#compile the parallel code for Tesla GPU
!cd ../../source_code/openacc && nvfortran -acc -ta=tesla,lineinfo -Minfo=accel -o rdf nvtx.f90 SOLUTION/rdf_data_directive.f90 -L/opt/nvidia/hpc_sdk/Linux_x86_64/21.3/cuda/11.2/lib64 -lnvToolsExt

In [ ]:
#profile and see output
!cd ../../source_code/openacc && nsys profile -t nvtx,openacc --stats=true --force-overwrite true -o rdf_parallel ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openacc/rdf_parallel.qdrep) and open it via the GUI.

### Further Optimization

Look at the profiler report from the previous section again. From the timeline, have a close look at the the kernel functions. Checkout the theoretical [occupancy](../GPU_Architecture_Terminologies.ipynb). As shown in the example screenshot below, `rdf_98_gpu` kernel has the theoretical occupancy of 62.5%. It clearly shows that the occupancy is a limiting factor. *Occupancy* is a measure of how well the GPU compute resources are being utilized. It is about how much parallelism is running / how much parallelism the hardware could run.

<img src="../images/f_data_thread.png">

NVIDIA GPUs are comprised of multiple [streaming multiprocessors (SMs)](../GPU_Architecture_Terminologies.ipynb) where it can manage up to 2048 concurrent threads (not actively running at the same time). Low occupancy shows that there are not enough active threads to fully utilize the computing resources. Higher occupancy implies that the scheduler has more active threads to choose from and hence achieves higher performance. So, what does this mean in OpenACC execution model?

**3 Levels of Parallelism: Gang, Worker, and Vector**
CUDA and OpenACC programming model use different terminologies for similar ideas. For example, in CUDA, parallel execution is organized into grids, blocks, and threads (checkout the [GPU Architecture Terminologies](../GPU_Architecture_Terminologies.ipynb) notebook to learn more about grids,blocks, threads). On the other hand, the OpenACC execution model has three levels of *Vector*, *Worker* and *Gang*. *Vector* threads work in lockstep, performing a single operation on multipler data (SIMD), whereas *Workers* computes one vector and *Gangs* have 1 or more workers and they all share resources such as cache, or SMs. Gangs run independent of each other.


OpenACC assumes the device has multiple processing elements (Streaming Multiprocessors on NVIDIA GPUs) running in parallel and mapping of OpenACC execution model on CUDA is as below:

- An OpenACC gang is a threadblock
- A worker is a warp
- An OpenACC vector is a CUDA thread

<img src="../images/diagram.png" width="50%" height="50%">

### Vector, Worker  and Gang Clauses

So, in order to improve the occupancy, we have to increase the parallelism within the gang. In other words, we have to increase the number of threads that can be scheduled on the GPU to improve GPU thread occupancy.

As you can see from the profiler report's screenshot, the grid dimension is fairly small `<53,1,1>` which shows the small amount of parallelism within the gang. We can use specific clauses to control the level of parallelism so that the compiler would use to parallelise the next loop. Each of *Vector*, *Worker* and *Gang* clauses can take a parameter to specify the size of each level of parallelism. For example we can control the vector length by using the `vector_length (num)` clause or we can add more workers by using `num_workers(num)`.

```fortran
!$acc parallel loop gang worker num_workers(32) vector_length(32)
do i=1,N
    !$acc loop vector
    do j=1,N
        ...
```

Now, add `gang`, `vector` and/or `worker` clauses to the code and experiment with the number of vectors and workers. Make necessary changes to the loop directives. Once done, save the file, re-compile via `make`, and profile it again. 

From the top menu, click on *File*, and *Open* `rdf.f90` from the current directory at `Fortran/source_code/openacc` directory. Remember to **SAVE** your code after changes, before running below cells.

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openacc && make clean && make

Let us start inspecting the compiler feedback and see if it applied the optimizations. Below is the screenshot of expected compiler feedback after adding the `gang`and `vector` clause to the code with vector length of 128. You can also change the vector length to 32 or 256 and see how the profiler output changes. The line 101 would change to `101, !$acc loop vector(256) ! threadidx%x` or `101, !$acc loop vector(32) ! threadidx%x`.

<img src="../images/f_gang_vector.png">

Now, validate the output by running the executable, and then **Profile** your code with Nsight Systems command line `nsys`.

In [ ]:
#Run on Nvidia GPU and check the output
!cd ../../source_code/openacc && ./rdf && cat Pair_entropy.dat

The output should be the following:

```
s2      :    -2.452690945278331     
s2bond  :    -24.37502820694527    
```

In [ ]:
#profile and see output
!cd ../../source_code/openacc && nsys profile -t nvtx,openacc --stats=true --force-overwrite true -o rdf_gang_vector ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openacc/rdf_gang_vector.qdrep) and open it via the GUI. Have a look at the example expected profiler report below:

<img src="../images/f_gang_128.png">

Checkout the kernel functions on the timeline and the occupancy. As you can see from the above screenshot, the theoretical occupancy is now 50% (slightly less than before). If we change the vector length to 32, the occupancy will not change as we do not have a lot of concurrent threads running.

<img src="../images/f_gang_32.png" width="30%" height="30%">

The loop iteration count inside the `rdf_98_gpu` function (line 98 according the above compiler feedback) is `natoms = 6720`. In this example, this number is the grid dimension of `<6720,1,1>`.


How much this optimization will speed-up the code will vary according to the application and the target accelerator, but it is not uncommon to see large speed-ups by using collapse on loop nests. 

Feel free to checkout the [solution](../../source_code/openacc/SOLUTION/rdf_gang_vector_length.f90) to help you understand better.

### Collapse Clauses
In order to expose more parallelism and improve the occupancy, we can use an additional clause called `collapse` in the `!$acc loop` to optimize loops. The loop directive gives the compiler additional information about the next loop in the source code through several clauses. Apply the `collapse(N)` clause to a loop directive to collapse the next `N` tightly-nested loops to be collapsed into a single, flattened loop. This is useful if you have many nested loops or when you have really short loops. Sample usage of collapse clause is given as follows:

```fortran
!$acc parallel loop collapse (2)
do i=1,N
    do j=1,N
        < loop code >
```

When the loop count in any of some tightly nested loops is relatively small compared to the available number of threads in the device, creating a single iteration space across all the nested loops, increases the iteration count thus allowing the compiler to extract more parallelism.

**Tips on where to use:**
- Collapse outer loops to enable creating more gangs.
- Collapse inner loops to enable longer vector lengths.
- Collapse all loops, when possible, to do both

Now, add `collapse` clause to the code and make necessary changes to the loop directives. Once done, save the file, re-compile via `make`, and profile it again. 

From the top menu, click on *File*, and *Open* `rdf.f90` from the current directory at `Fortran/source_code/openacc` directory. Remember to **SAVE** your code after changes, before running below cells.

In [ ]:
#compile for Tesla GPU
!cd ../../source_code/openacc && make clean && make

Let us start inspecting the compiler feedback and see if it applied the optimizations. Below is the screenshot of expected compiler feedback after adding the `collapse`clause to the code. You can see that nested loops on line 184 has been successfully collapsed.

<img src="../images/f_collapse_feedback.png">

Now, validate the output by running the executable, and then **Profile** your code with Nsight Systems command line `nsys`.

In [ ]:
#Run on Nvidia GPU and check the output
!cd ../../source_code/openacc && ./rdf && cat Pair_entropy.dat

The output should be the following:

```
s2      :    -2.452690945278331     
s2bond  :    -24.37502820694527    
```

In [ ]:
#profile and see output
!cd ../../source_code/openacc && nsys profile -t nvtx,openacc --stats=true --force-overwrite true -o rdf_collapse ./rdf

Let's checkout the profiler's report. [Download the profiler output](../../source_code/openacc/rdf_collapse.qdrep) and open it via the GUI. Have a look at the example expected profiler report below:

<img src="../images/f_collapse_thread.png">

Checkout the kernel functions on the timeline and the occupancy. As you can see from the above screenshot, the theoretical occupancy is now 62.5%.

The iteration count for the collapsed loop is `natoms * natoms` where `natoms = 6720`  (in this example). So, the iteration count for this particular loop (collapse loop) inside the `rdf_98_gpu` function is 45158400 and this number divided by the vector length of *128* is 352800. The maximum grid size is 65535 blocks in each dimension and we can see that we have a grid dimension of `<65535,1,1>` in this example.

By creating a single iteration space across the nested loops and increasing the iteration count, we improved the occupancy and extracted more parallelism.

**Notes:**
- 100% occupancy is not required for, nor does it guarantee best performance.
- Less than 50% occupancy is often a red flag

How much this optimization will speed-up the code will vary according to the application and the target accelerator, but it is not uncommon to see large speed-ups by using collapse on loop nests. 

Feel free to checkout the [solution](../../source_code/openacc/SOLUTION/rdf_collapse.f90) to help you understand better.


There is another clause which may be useful in optimizing loops. *Tile* clause breaks down the next loops into tiles before parallelising and it promotes data locality as the device can then use data from nearby tiles. 

```fortran
!$acc parallel loop tile (4,4)
do i=1,N
    do j=1,N
        < loop code >
```

We do not cover this in the labs but this is something you can explore and compare with the explained methods of loop optimization.

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommend you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f nways_files.zip
zip -r nways_files.zip *

**After** executing the above zip command, you should be able to download the zip file [here](../nways_files.zip). Let us now go back to parallelizing our code using other approaches.

**IMPORTANT**: If you wish to dig deeper and profile the kernel with the Nsight Computer profiler, go to the next notebook. Otherwise, please click on **HOME** to go back to the main notebook for *N ways of GPU programming for MD* code. 

-----

# <p style="text-align:center;border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em"> <a href=../../../nways_MD_start.ipynb>HOME</a>&nbsp; &nbsp; &nbsp; <a href=nways_openacc_opt_2.ipynb>NEXT</a></p>

-----


# Links and Resources

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[NVIDIA Nsight Compute](https://developer.nvidia.com/nsight-compute)

[CUDA Toolkit Download](https://developer.nvidia.com/cuda-downloads)

**NOTE**: To be able to see the Nsight System profiler output, please download Nsight System latest version from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 

## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).